In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
from bs4 import BeautifulSoup
from graphviz import Digraph
from PIL import Image, ImageTk
import os

# Clase principal del editor de HTML
class HTMLEditor(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("HTML EDITOR PROYECTO PROGRAMACION")
        self.geometry("1200x600")
        
        self.filename = None
        
        # Menú principal
        self.menu = tk.Menu(self)
        self.config(menu=self.menu)
        
        # Menú de archivo
        self.file_menu = tk.Menu(self.menu, tearoff=0)
        self.menu.add_cascade(label="Archivo", menu=self.file_menu)
        self.file_menu.add_command(label="Nuevo", command=self.new_file)
        self.file_menu.add_command(label="Abrir", command=self.open_file)
        self.file_menu.add_command(label="Guardar", command=self.save_file)
        self.file_menu.add_command(label="Guardar como", command=self.save_as_file)
        self.file_menu.add_separator()
        self.file_menu.add_command(label="Salir", command=self.exit_program)
        
        # Menú de edición
        self.edit_menu = tk.Menu(self.menu, tearoff=0)
        self.menu.add_cascade(label="Editar", menu=self.edit_menu)
        self.edit_menu.add_command(label="Buscar", command=self.find_text)
        self.edit_menu.add_command(label="Remplazar", command=self.replace_text)
        self.edit_menu.add_command(label="Ir a", command=self.go_to_line)
        
        # Área de texto para editar HTML (lado izquierdo)
        self.text_frame = tk.Frame(self)
        self.text_frame.pack(side='left', fill='both', expand=True)
        
        self.line_numbers = tk.Text(self.text_frame, width=4, padx=3, takefocus=0, border=0, background='lightgrey', state='disabled', wrap='none')
        self.line_numbers.pack(side='left', fill='y')
        
        self.text_area = tk.Text(self.text_frame, wrap='word', undo=True)
        self.text_area.pack(expand=1, fill='both', side='left')
        
        self.scrollbar = tk.Scrollbar(self.text_frame, command=self.text_area.yview)
        self.scrollbar.pack(side='right', fill='y')
        
        self.text_area.config(yscrollcommand=self.scrollbar.set)
        self.text_area.bind('<KeyRelease>', self.on_key_release)
        
        # Palabras clave de HTML para resaltar
        self.html_keywords = [
            "<!DOCTYPE", "<a", "<abbr", "<acronym", "<address", "<applet", "<area", "<article", "<aside>", "<audio>",
            "<b>", "<base>", "<basefont>", "<bdi>", "<bdo>", "<big>", "<blockquote>", "<body>", "<br>", "<button>",
            "<canvas>", "<caption>", "<center>", "<cite>", "<code>", "<col>", "<colgroup>", "<data>", "<datalist>", "<dd>",
            "<del>", "<details>", "<dfn>", "<dialog>", "<div>", "<dl>", "<dt>", "<em>", "<embed>", "<fieldset>", "<figcaption>",
            "<figure>", "<font>", "<footer>", "<form>", "<frame>", "<frameset>", "<h1>", "<h2>", "<h3>", "<h4>", "<h5>", "<h6>",
            "<head>", "<header>", "<hr>", "<html>", "<i>", "<iframe>", "<img>", "<input>", "<ins>", "<kbd>", "<label>", "<legend>",
            "<li>", "<link>", "<main>", "<map>", "<mark>", "<meta>", "<meter>", "<nav>", "<noframes>", "<noscript>", "<object>",
            "<ol>", "<optgroup>", "<option>", "<output>", "<p>", "<param>", "<picture>", "<pre>", "<progress>", "<q>",
            "<rp>", "<rt>", "<ruby>", "<s>", "<samp>", "<script>", "<section>", "<select>", "<small>", "<source>", "<span>",
            "<strike>", "<strong>", "<style>", "<sub>", "<summary>", "<sup>", "<svg>", "<table>", "<tbody>", "<td>", "<template>",
            "<textarea>", "<tfoot>", "<th>", "<thead>", "<time>", "<title>", "<tr>", "<track>", "<tt>", "<u>", "<ul>", "<var>",
            "<video>", "<wbr", "</a>", "</abbr>", "</acronym>", "</address>", "</applet>", "</area>", "</article>", "</aside>", "</audio>",
            "</b>", "</base>", "</basefont>", "</bdi>", "</bdo>", "</big>", "</blockquote>", "</body>", "</br>", "</button>",
            "</canvas>", "</caption>", "</center>", "</cite>", "</code>", "</col>", "</colgroup>", "</data>", "</datalist>", "</dd>",
            "</del>", "</details>", "</dfn>", "</dialog>", "</div>", "</dl>", "</dt>", "</em>", "</embed>", "</fieldset>", "</figcaption>",
            "</figure>", "</font>", "</footer>", "</form>", "</frame>", "</frameset>", "</h1>", "</h2>", "</h3>", "</h4>", "</h5>", "</h6>",
            "</head>", "</header>", "</hr>", "</html>", "</i>", "</iframe>", "</img>", "</input>", "</ins>", "</kbd>", "</label>", "</legend>",
            "</li>", "</link>", "</main>", "</map>", "</mark>", "</meta>", "</meter>", "</nav>", "</noframes>", "</noscript>", "</object>",
            "</ol>", "</optgroup>", "</option>", "</output>", "</p>", "</param>", "</picture>", "</pre>", "</progress>", "</q>",
            "</rp>", "</rt>", "</ruby>", "</s>", "</samp>", "</script>", "</section>", "</select>", "</small>", "</source>", "</span>",
            "</strike>", "</strong>", "</style>", "</sub>", "</summary>", "</sup>", "</svg>", "</table>", "</tbody>", "</td>", "</template>",
            "</textarea>", "</tfoot>", "</th>", "</thead>", "</time>", "</title>", "</tr>", "</track>", "</tt>", "</u>", "</ul>", "</var>",
            "</video>", "</wbr"
        ]
        
        # Lado derecho para la visualización del DOM
        self.canvas_frame = tk.Frame(self)
        self.canvas_frame.pack(side='right', fill='both', expand=True)
        
        self.canvas = tk.Canvas(self.canvas_frame, bg='white')
        self.canvas.pack(expand=1, fill='both')
        
        self.update_line_numbers()
        self.highlight_html_keywords()

    # Evento al liberar una tecla
    def on_key_release(self, event=None):
        self.update_line_numbers()
        self.highlight_html_keywords()
        self.update_dom_tree()

    # Actualiza los números de línea
    def update_line_numbers(self):
        self.line_numbers.config(state='normal')
        self.line_numbers.delete('1.0', 'end')
        
        line_numbers_str = "\n".join(map(str, range(1, int(self.text_area.index('end').split('.')[0]))))
        self.line_numbers.insert('1.0', line_numbers_str)
        
        self.line_numbers.config(state='disabled')

    # Resalta palabras clave de HTML en el área de texto
    def highlight_html_keywords(self):
        self.text_area.tag_remove("keyword", "1.0", tk.END)
        for keyword in self.html_keywords:
            start_pos = "1.0"
            while True:
                start_pos = self.text_area.search(keyword, start_pos, stopindex=tk.END)
                if not start_pos:
                    break
                end_pos = f"{start_pos}+{len(keyword)}c"
                self.text_area.tag_add("keyword", start_pos, end_pos)
                start_pos = end_pos
        self.text_area.tag_config("keyword", foreground="red")

    # Actualiza la visualización del árbol DOM
    def update_dom_tree(self):
        html_content = self.text_area.get(1.0, tk.END)
        soup = BeautifulSoup(html_content, 'html.parser')
        
        dot = Digraph()
        
        # Función para dibujar nodos en el gráfico
        def draw_node(dot, node, parent_name=None):
            if isinstance(node, str):
                if not node.strip():
                    return
                node_name = node.strip()
                shape = 'ellipse'
            else:
                node_name = node.name
                shape = 'box'
            
            dot.node(node_name, node_name, shape=shape)
            
            if parent_name:
                dot.edge(parent_name, node_name)
            
            if hasattr(node, 'contents'):
                for child in node.contents:
                    draw_node(dot, child, node_name)
        
        draw_node(dot, soup)
        
        dot.render('dom_tree', format='png', cleanup=False)
        self.display_image('dom_tree.png')

    # Muestra la imagen generada en el lienzo
    def display_image(self, path):
        img = Image.open(path)
        img = img.resize((400, 400), Image.LANCZOS)
        self.img_tk = ImageTk.PhotoImage(img)
        
        self.canvas.create_image(0, 0, anchor='nw', image=self.img_tk)
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))

    # Crear un nuevo archivo
    def new_file(self):
        self.text_area.delete(1.0, tk.END)
        self.filename = None

    # Abrir un archivo existente
    def open_file(self):
        self.filename = filedialog.askopenfilename(defaultextension=".html", filetypes=[("HTML files", "*.html"), ("All files", "*.*")])
        if self.filename:
            with open(self.filename, "r") as file:
                self.text_area.delete(1.0, tk.END)
                self.text_area.insert(1.0, file.read())
            self.update_dom_tree()

    # Guardar el archivo actual
    def save_file(self):
        if self.filename:
            with open(self.filename, "w") as file:
                file.write(self.text_area.get(1.0, tk.END))
        else:
            self.save_as_file()

    # Guardar el archivo actual con un nuevo nombre
    def save_as_file(self):
        self.filename = filedialog.asksaveasfilename(defaultextension=".html", filetypes=[("HTML files", "*.html"), ("All files", "*.*")])
        if self.filename:
            with open(self.filename, "w") as file:
                file.write(self.text_area.get(1.0, tk.END))

    # Buscar texto en el archivo
    def find_text(self):
        find_window = tk.Toplevel(self)
        find_window.title("Buscar texto")
        find_window.transient(self)
        
        tk.Label(find_window, text="Buscar:").grid(row=0, column=0, padx=4, pady=4)
        find_entry = tk.Entry(find_window, width=25)
        find_entry.grid(row=0, column=1, padx=4, pady=4)
        find_entry.focus_set()
        
        def find():
            self.text_area.tag_remove("found", "1.0", tk.END)
            target = find_entry.get()
            if target:
                idx = "1.0"
                while True:
                    idx = self.text_area.search(target, idx, nocase=1, stopindex=tk.END)
                    if not idx:
                        break
                    lastidx = f"{idx}+{len(target)}c"
                    self.text_area.tag_add("found", idx, lastidx)
                    idx = lastidx
                self.text_area.tag_config("found", foreground="red", background="yellow")
                find_window.after(5000, lambda: self.text_area.tag_remove("found", "1.0", tk.END))  # Remover fondo amarillo después de 5 segundos
        
        tk.Button(find_window, text="Buscar todos", command=find).grid(row=0, column=2, padx=4, pady=4)

    # Remplazar texto en el archivo
    def replace_text(self):
        replace_window = tk.Toplevel(self)
        replace_window.title("Remplazar texto")
        replace_window.transient(self)
        
        tk.Label(replace_window, text="Buscar:").grid(row=0, column=0, padx=4, pady=4)
        find_entry = tk.Entry(replace_window, width=25)
        find_entry.grid(row=0, column=1, padx=4, pady=4)
        find_entry.focus_set()
        
        tk.Label(replace_window, text="Remplazar:").grid(row=1, column=0, padx=4, pady=4)
        replace_entry = tk.Entry(replace_window, width=25)
        replace_entry.grid(row=1, column=1, padx=4, pady=4)
        
        def replace():
            self.text_area.tag_remove("found", "1.0", tk.END)
            target = find_entry.get()
            replacement = replace_entry.get()
            if target and replacement:
                idx = "1.0"
                while True:
                    idx = self.text_area.search(target, idx, nocase=1, stopindex=tk.END)
                    if not idx:
                        break
                    lastidx = f"{idx}+{len(target)}c"
                    self.text_area.delete(idx, lastidx)
                    self.text_area.insert(idx, replacement)
                    lastidx = f"{idx}+{len(replacement)}c"
                    self.text_area.tag_add("found", idx, lastidx)
                    idx = lastidx
                self.text_area.tag_config("found", foreground="red", background="yellow")
                replace_window.after(5000, lambda: self.text_area.tag_remove("found", "1.0", tk.END))  # Remover fondo amarillo después de 5 segundos
        
        tk.Button(replace_window, text="Remplazar todos", command=replace).grid(row=2, column=1, padx=4, pady=4)

    # Ir a una línea específica en el archivo
    def go_to_line(self):
        line_number = simpledialog.askinteger("Ir a", "la línea número:")
        if line_number:
            self.text_area.mark_set("insert", f"{line_number}.0")
            self.text_area.see("insert")

    # Salir del programa
    def exit_program(self):
        if messagebox.askokcancel("Salir", "¿Estás seguro de salir?"):
            self.destroy()

# Punto de entrada de la aplicación
if __name__ == "__main__":
    app = HTMLEditor()
    app.mainloop()
